In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import files
from scipy import stats
from scipy.stats import t, f

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from scipy.stats import skew, kurtosis

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
random_seed = 2
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [ ]:
data=pd.read_csv('teams.csv')

In [ ]:
df=data['dysfunc']
xdf=pd.DataFrame({'a':df-0.05,'b':df,'c':df+0.05})
x=xdf.to_numpy()
net=data['negtone']
mdf=pd.DataFrame({'a':net-0.05,'b':net,'c':net+0.05})
m=mdf.to_numpy()
per=data['perform']
ydf=pd.DataFrame({'a':per-0.05,'b':per,'c':per+0.05})
y=ydf.to_numpy()


In [ ]:
def gpbdl1(data):
  df=data['dysfunc']
  xdf=pd.DataFrame({'a':df-0.05,'b':df,'c':df+0.05})
  x=xdf.to_numpy()
  net=data['negtone']
  mdf=pd.DataFrame({'a':net-0.05,'b':net,'c':net+0.05})
  m=mdf.to_numpy()
  per=data['perform']
  ydf=pd.DataFrame({'a':per-0.05,'b':per,'c':per+0.05})
  y=ydf.to_numpy()
  X_l = []
  X_m = []
  X_r = []

  M_l = []
  M_m = []
  M_r = []

  Y_l = []
  Y_m = []
  Y_r = []
  for i in range(len(data)):
    X_l.append(x[i][0])
    X_m.append(x[i][1])
    X_r.append(x[i][2])
  for i in range(len(data)):
    M_l.append(m[i][0])
    M_m.append(m[i][1])
    M_r.append(m[i][2])

  for i in range(len(data)):
    Y_l.append(y[i][0])
    Y_m.append(y[i][1])
    Y_r.append(y[i][2])


  ep = 800
  x_train = torch.FloatTensor([X_l])
  x_train = x_train.T
  y_train = torch.FloatTensor([M_l])
  y_train = y_train.T
  Weight_l = torch.zeros((1,1), requires_grad=True)  # 가중치 초기화
  Bias_l = torch.zeros(1, requires_grad=True)
  w_l=torch.zeros((9,1))
  b_l=torch.zeros((9,1))
  i=0
  # optimizer 설정
  optimizer = optim.Adagrad([Weight_l, Bias_l], lr=0.01)  # Adam, lr - 0.01고정

  nb_epochs = ep  # epoch

  for epoch in range(nb_epochs + 1):

    # H(X) 계산
    hypothesis = x_train.matmul(Weight_l) + Bias_l

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # Backpropagation
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 ==0:
      w_l[i]= Weight_l
      b_l[i]= Bias_l
      i=i+1


  x_train = torch.FloatTensor([X_m])
  x_train = x_train.T
  x_train.shape
  y_train = torch.FloatTensor([M_m]) # M
  y_train = y_train.T
  Weight_m = torch.zeros((1,1), requires_grad=True)  # 가중치 초기화
  Bias_m = torch.zeros(1, requires_grad=True)
  w_m=torch.zeros((9,1))
  b_m=torch.zeros((9,1))
  i=0
  optimizer = optim.Adagrad([Weight_m, Bias_m], lr=0.01)  # Adam, lr - 0.01고정


  for epoch in range(nb_epochs + 1):

    # H(X) 계산
    hypothesis = x_train.matmul(Weight_m) + Bias_m

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # Backpropagation
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 ==0:
      w_m[i]= Weight_m
      b_m[i]= Bias_m
      i=i+1

  x_train = torch.FloatTensor([X_r])
  x_train = x_train.T
  x_train.shape
  y_train = torch.FloatTensor([M_r]) # M
  y_train = y_train.T
  Weight_r = torch.zeros((1,1), requires_grad=True)  # 가중치 초기화
  Bias_r = torch.zeros(1, requires_grad=True)
  w_r=torch.zeros((9,1))
  b_r=torch.zeros((9,1))
  i=0
  optimizer = optim.Adagrad([Weight_r, Bias_r], lr=0.01)  # Adam, lr - 0.01고정

  for epoch in range(nb_epochs + 1):

    # H(X) 계산
    hypothesis = x_train.matmul(Weight_r) + Bias_r

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # Backpropagation
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 ==0:
      w_r[i]= Weight_r
      b_r[i]= Bias_r
      i=i+1

  w1=(w_l + w_m + w_r)/ 3
  b1=(b_l + b_m + b_r)/ 3


  x_train = torch.FloatTensor([X_l,M_l])
  x_train = x_train.T
  y_train = torch.FloatTensor([Y_l])
  y_train = y_train.T
  Weight_l = torch.zeros((2,1), requires_grad=True)  # 가중치 초기화
  Bias_l = torch.zeros(1, requires_grad=True)
  w_l=torch.zeros((9,2))
  b_l=torch.zeros((9,1))
  i=0
  # optimizer 설정
  optimizer = optim.Adagrad([Weight_l, Bias_l], lr=0.01)  # Adam, lr - 0.01고정

  nb_epochs = ep  # epoch

  for epoch in range(nb_epochs + 1):

    # H(X) 계산
    hypothesis = x_train.matmul(Weight_l) + Bias_l

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # Backpropagation
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 ==0:
      w_l[i][0]= Weight_l[0]
      w_l[i][1]= Weight_l[1]
      b_l[i]= Bias_l
      i=i+1


  x_train = torch.FloatTensor([X_m,M_m])
  x_train = x_train.T
  x_train.shape
  y_train = torch.FloatTensor([Y_m]) # M
  y_train = y_train.T
  Weight_m = torch.zeros((2,1), requires_grad=True)  # 가중치 초기화
  Bias_m = torch.zeros(1, requires_grad=True)
  w_m=torch.zeros((9,2))
  b_m=torch.zeros((9,1))
  i=0
  optimizer = optim.Adagrad([Weight_m, Bias_m], lr=0.01)  # Adam, lr - 0.01고정


  for epoch in range(nb_epochs + 1):

    # H(X) 계산
    hypothesis = x_train.matmul(Weight_m) + Bias_m

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # Backpropagation
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 ==0:
      w_m[i][0]= Weight_m[0]
      w_m[i][1]= Weight_m[1]
      b_m[i]= Bias_m
      i=i+1

  x_train = torch.FloatTensor([X_r,M_r])
  x_train = x_train.T
  x_train.shape
  y_train = torch.FloatTensor([Y_r]) # M
  y_train = y_train.T
  Weight_r = torch.zeros((2,1), requires_grad=True)  # 가중치 초기화
  Bias_r = torch.zeros(1, requires_grad=True)
  w_r=torch.zeros((9,2))
  b_r=torch.zeros((9,1))
  i=0
  optimizer = optim.Adagrad([Weight_r, Bias_r], lr=0.01)  # Adam, lr - 0.01고정

  for epoch in range(nb_epochs + 1):

    # H(X) 계산
    hypothesis = x_train.matmul(Weight_r) + Bias_r

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # Backpropagation
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 ==0:
      w_r[i][0]= Weight_r[0]
      w_r[i][1]= Weight_r[1]
      b_r[i]= Bias_r
      i=i+1

  w2=(w_l + w_m + w_r)/ 3
  b2=(b_l + b_m + b_r)/ 3

  x_train = torch.FloatTensor([X_l])
  x_train = x_train.T
  y_train = torch.FloatTensor([Y_l])
  y_train = y_train.T
  Weight_l = torch.zeros((1,1), requires_grad=True)  # 가중치 초기화
  Bias_l = torch.zeros(1, requires_grad=True)
  w_l=torch.zeros((9,1))
  b_l=torch.zeros((9,1))
  i=0
  # optimizer 설정
  optimizer = optim.Adagrad([Weight_l, Bias_l], lr=0.01)  # Adam, lr - 0.01고정

  for epoch in range(nb_epochs + 1):

    # H(X) 계산
    hypothesis = x_train.matmul(Weight_l) + Bias_l

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # Backpropagation
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 ==0:
      w_l[i]= Weight_l
      b_l[i]= Bias_l
      i=i+1


  x_train = torch.FloatTensor([X_m])
  x_train = x_train.T
  y_train = torch.FloatTensor([Y_m])
  y_train = y_train.T
  Weight_m = torch.zeros((1,1), requires_grad=True)  # 가중치 초기화
  Bias_m = torch.zeros(1, requires_grad=True)
  w_m=torch.zeros((9,1))
  b_m=torch.zeros((9,1))
  i=0
  optimizer = optim.Adagrad([Weight_m, Bias_m], lr=0.01)  # Adam, lr - 0.01고정
  for epoch in range(nb_epochs + 1):

    # H(X) 계산ssssssssssssssssssssssssssss
    hypothesis = x_train.matmul(Weight_m) + Bias_m

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # Backpropagation
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 ==0:
      w_m[i]= Weight_m
      b_m[i]= Bias_m
      i=i+1

  x_train = torch.FloatTensor([X_r])
  x_train = x_train.T
  y_train = torch.FloatTensor([Y_r])
  y_train = y_train.T
  Weight_r = torch.zeros((1,1), requires_grad=True)  # 가중치 초기화
  Bias_r = torch.zeros(1, requires_grad=True)
  w_r=torch.zeros((9,1))
  b_r=torch.zeros((9,1))
  i=0
  optimizer = optim.Adagrad([Weight_r, Bias_r], lr=0.01)  # Adam, lr - 0.01고정
  for epoch in range(nb_epochs + 1):

    # H(X) 계산
    hypothesis = x_train.matmul(Weight_r) + Bias_r

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # Backpropagation
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
  if epoch % 100 ==0:
      w_r[i]= Weight_r
      b_r[i]= Bias_r
      i=i+1

  w3=(w_l + w_m + w_r)/ 3
  b3=(b_l + b_m + b_r)/ 3
  return w1,b1,w2,b2,w3,b3

In [ ]:
av=gpbdl1(data)
av

(tensor([[0.0100],
         [0.1743],
         [0.2436],
         [0.2934],
         [0.3328],
         [0.3654],
         [0.3931],
         [0.4170],
         [0.4379]], grad_fn=<DivBackward0>),
 tensor([[0.0033],
         [0.0415],
         [0.0393],
         [0.0375],
         [0.0360],
         [0.0349],
         [0.0339],
         [0.0330],
         [0.0322]], grad_fn=<DivBackward0>),
 tensor([[ 0.0100, -0.0100],
         [ 0.1626, -0.1742],
         [ 0.2199, -0.2420],
         [ 0.2584, -0.2897],
         [ 0.2873, -0.3266],
         [ 0.3101, -0.3564],
         [ 0.3288, -0.3811],
         [ 0.3443, -0.4019],
         [ 0.3574, -0.4196]], grad_fn=<DivBackward0>),
 tensor([[-0.0033],
         [-0.0290],
         [-0.0280],
         [-0.0270],
         [-0.0263],
         [-0.0256],
         [-0.0251],
         [-0.0247],
         [-0.0243]], grad_fn=<DivBackward0>),
 tensor([[0.0434],
         [0.0702],
         [0.0733],
         [0.0735],
         [0.0735],
         [0.0735],

In [ ]:
a_b1=[None]*5000
c_b1=[None]*5000
b_b1=[None]*5000
cp_b1=[None]*5000

a_b2=[None]*5000
c_b2=[None]*5000
b_b2=[None]*5000
cp_b2=[None]*5000
a_b3=[None]*5000
c_b3=[None]*5000
b_b3=[None]*5000
cp_b3=[None]*5000
a_b4=[None]*5000
c_b4=[None]*5000
b_b4=[None]*5000
cp_b4=[None]*5000
a_b5=[None]*5000
c_b5=[None]*5000
b_b5=[None]*5000
cp_b5=[None]*5000
a_b6=[None]*5000
c_b6=[None]*5000
b_b6=[None]*5000
cp_b6=[None]*5000
a_b7=[None]*5000
c_b7=[None]*5000
b_b7=[None]*5000
cp_b7=[None]*5000
a_b8=[None]*5000
c_b8=[None]*5000
b_b8=[None]*5000
cp_b8=[None]*5000
a_b9=[None]*5000
c_b9=[None]*5000
b_b9=[None]*5000
cp_b9=[None]*5000

bi1=[None]*5000
bi2=[None]*5000
bi3=[None]*5000
bi4=[None]*5000
bi5=[None]*5000
bi6=[None]*5000
bi7=[None]*5000
bi8=[None]*5000
bi9=[None]*5000
fbi1=[None]*5000
fbi2=[None]*5000
fbi3=[None]*5000
fbi4=[None]*5000
fbi5=[None]*5000
fbi6=[None]*5000
fbi7=[None]*5000
fbi8=[None]*5000
fbi9=[None]*5000

sbi1=[None]*5000
sbi2=[None]*5000
sbi3=[None]*5000
sbi4=[None]*5000
sbi5=[None]*5000
sbi6=[None]*5000
sbi7=[None]*5000
sbi8=[None]*5000
sbi9=[None]*5000


In [ ]:
from tqdm import tqdm

In [ ]:
#부트스트랩 표본
cprime_boot=[None]*5000
for i in tqdm(range(0,5000)):
  new_data=data.sample(n=60,replace=True,random_state=i)
  parameters=gpbdl1(new_data)
  a_b1[i]= parameters[0][0].detach()
  c_b1[i]= parameters[2][0][0].detach()
  b_b1[i]= parameters[2][0][1].detach()
  cp_b1[i]= parameters[4][0].detach()
  a_b2[i]= parameters[0][1].detach()
  c_b2[i]= parameters[2][1][0].detach()
  b_b2[i]= parameters[2][1][1].detach()
  cp_b2[i]= parameters[4][1].detach()
  a_b3[i]= parameters[0][2].detach()
  c_b3[i]= parameters[2][2][0].detach()
  b_b3[i]= parameters[2][2][1].detach()
  cp_b3[i]= parameters[4][2].detach()
  a_b4[i]= parameters[0][3].detach()
  c_b4[i]= parameters[2][3][0].detach()
  b_b4[i]= parameters[2][3][1].detach()
  cp_b4[i]= parameters[4][3].detach()
  a_b5[i]= parameters[0][4].detach()
  c_b5[i]= parameters[2][4][0].detach()
  b_b5[i]= parameters[2][4][1].detach()
  cp_b5[i]= parameters[4][4].detach()
  a_b6[i]= parameters[0][5].detach()
  c_b6[i]= parameters[2][5][0].detach()
  b_b6[i]= parameters[2][5][1].detach()
  cp_b6[i]= parameters[4][5].detach()
  a_b7[i]= parameters[0][6].detach()
  c_b7[i]= parameters[2][6][0].detach()
  b_b7[i]= parameters[2][6][1].detach()
  cp_b7[i]= parameters[4][6].detach()
  a_b8[i]= parameters[0][7].detach()
  c_b8[i]= parameters[2][7][0].detach()
  b_b8[i]= parameters[2][7][1].detach()
  cp_b8[i]= parameters[4][7].detach()
  a_b9[i]= parameters[0][8].detach()
  c_b9[i]= parameters[2][8][0].detach()
  b_b9[i]= parameters[2][8][1].detach()
  cp_b9[i]= parameters[4][8].detach()

  bi1[i]=parameters[3][0].detach()
  bi2[i]=parameters[3][1].detach()
  bi3[i]=parameters[3][2].detach()
  bi4[i]=parameters[3][3].detach()
  bi5[i]=parameters[3][4].detach()
  bi6[i]=parameters[3][5].detach()
  bi7[i]=parameters[3][6].detach()
  bi8[i]=parameters[3][7].detach()
  bi9[i]=parameters[3][8].detach()

  fbi1[i]=parameters[1][0].detach()
  fbi2[i]=parameters[1][1].detach()
  fbi3[i]=parameters[1][2].detach()
  fbi4[i]=parameters[1][3].detach()
  fbi5[i]=parameters[1][4].detach()
  fbi6[i]=parameters[1][5].detach()
  fbi7[i]=parameters[1][6].detach()
  fbi8[i]=parameters[1][7].detach()
  fbi9[i]=parameters[1][8].detach()

  sbi1[i]=parameters[5][0].detach()
  sbi2[i]=parameters[5][1].detach()
  sbi3[i]=parameters[5][2].detach()
  sbi4[i]=parameters[5][3].detach()
  sbi5[i]=parameters[5][4].detach()
  sbi6[i]=parameters[5][5].detach()
  sbi7[i]=parameters[5][6].detach()
  sbi8[i]=parameters[5][7].detach()
  sbi9[i]=parameters[5][8].detach()

 44%|████▎     | 2180/5000 [1:43:25<2:13:47,  2.85s/it]


KeyboardInterrupt: ignored

In [ ]:
np.save('a_b1',a_b1)
np.save('b_b1',b_b1)
np.save('c_b1',c_b1)
np.save('cp_b1',cp_b1)
np.save('a_b2',a_b2)
np.save('b_b2',b_b2)
np.save('c_b2',c_b2)
np.save('cp_b2',cp_b2)
np.save('a_b3',a_b3)
np.save('b_b3',b_b3)
np.save('c_b3',c_b3)
np.save('cp_b3',cp_b3)
np.save('a_b4',a_b4)
np.save('b_b4',b_b4)
np.save('c_b4',c_b4)
np.save('cp_b4',cp_b4)
np.save('a_b5',a_b5)
np.save('b_b5',b_b5)
np.save('c_b5',c_b5)
np.save('cp_b5',cp_b5)
np.save('a_b6',a_b6)
np.save('b_b6',b_b6)
np.save('c_b6',c_b6)
np.save('cp_b6',cp_b6)
np.save('a_b7',a_b7)
np.save('b_b7',b_b7)
np.save('c_b7',c_b7)
np.save('cp_b7',cp_b7)
np.save('a_b8',a_b8)
np.save('b_b8',b_b8)
np.save('c_b8',c_b8)
np.save('cp_b8',cp_b8)
np.save('a_b9',a_b9)
np.save('b_b9',b_b9)
np.save('c_b9',c_b9)
np.save('cp_b9',cp_b9)

np.save('bi1',bi1)
np.save('bi2',bi2)
np.save('bi3',bi3)
np.save('bi4',bi4)
np.save('bi5',bi5)
np.save('bi6',bi6)
np.save('bi7',bi7)
np.save('bi8',bi8)
np.save('bi9',bi9)
np.save('fbi1',fbi1)
np.save('fbi2',fbi2)
np.save('fbi3',fbi3)
np.save('fbi4',fbi4)
np.save('fbi5',fbi5)
np.save('fbi6',fbi6)
np.save('fbi7',fbi7)
np.save('fbi8',fbi8)
np.save('fbi9',fbi9)

np.save('sbi1',sbi1)
np.save('sbi2',sbi2)
np.save('sbi3',sbi3)
np.save('sbi4',sbi4)
np.save('sbi5',sbi5)
np.save('sbi6',sbi6)
np.save('sbi7',sbi7)
np.save('sbi8',sbi8)
np.save('sbi9',sbi9)

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:521: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.asanyarray(arr)
/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
a_b1=np.load('a_b1.npy',allow_pickle=True)
b_b1=np.load('b_b1.npy',allow_pickle=True)
c_b1=np.load('c_b1.npy',allow_pickle=True)
cp_b1=np.load('cp_b1.npy',allow_pickle=True)
ab_b1=a_b1*b_b1
a_b2=np.load('a_b2.npy',allow_pickle=True)
b_b2=np.load('b_b2.npy',allow_pickle=True)
c_b2=np.load('c_b2.npy',allow_pickle=True)
cp_b2=np.load('cp_b2.npy',allow_pickle=True)
ab_b2=a_b2*b_b2
a_b3=np.load('a_b3.npy',allow_pickle=True)
b_b3=np.load('b_b3.npy',allow_pickle=True)
c_b3=np.load('c_b3.npy',allow_pickle=True)
cp_b3=np.load('cp_b3.npy',allow_pickle=True)
ab_b3=a_b3*b_b3
a_b4=np.load('a_b4.npy',allow_pickle=True)
b_b4=np.load('b_b4.npy',allow_pickle=True)
c_b4=np.load('c_b4.npy',allow_pickle=True)
cp_b4=np.load('cp_b4.npy',allow_pickle=True)
ab_b4=a_b4*b_b4
a_b5=np.load('a_b5.npy',allow_pickle=True)
b_b5=np.load('b_b5.npy',allow_pickle=True)
c_b5=np.load('c_b5.npy',allow_pickle=True)
cp_b5=np.load('cp_b5.npy',allow_pickle=True)
ab_b5=a_b5*b_b5
a_b6=np.load('a_b6.npy',allow_pickle=True)
b_b6=np.load('b_b6.npy',allow_pickle=True)
c_b6=np.load('c_b6.npy',allow_pickle=True)
cp_b6=np.load('cp_b6.npy',allow_pickle=True)
ab_b6=a_b6*b_b6
a_b7=np.load('a_b7.npy',allow_pickle=True)
b_b7=np.load('b_b7.npy',allow_pickle=True)
c_b7=np.load('c_b7.npy',allow_pickle=True)
cp_b7=np.load('cp_b7.npy',allow_pickle=True)
ab_b7=a_b7*b_b7
a_b8=np.load('a_b8.npy',allow_pickle=True)
b_b8=np.load('b_b8.npy',allow_pickle=True)
c_b8=np.load('c_b8.npy',allow_pickle=True)
cp_b8=np.load('cp_b8.npy',allow_pickle=True)
ab_b8=a_b8*b_b8
a_b9=np.load('a_b9.npy',allow_pickle=True)
b_b9=np.load('b_b9.npy',allow_pickle=True)
c_b9=np.load('c_b9.npy',allow_pickle=True)
cp_b9=np.load('cp_b9.npy',allow_pickle=True)
ab_b9=a_b9*b_b9

bi1=np.load('bi1.npy',allow_pickle=True)
bi2=np.load('bi2.npy',allow_pickle=True)
bi3=np.load('bi3.npy',allow_pickle=True)
bi4=np.load('bi4.npy',allow_pickle=True)
bi5=np.load('bi5.npy',allow_pickle=True)
bi6=np.load('bi6.npy',allow_pickle=True)
bi7=np.load('bi7.npy',allow_pickle=True)
bi8=np.load('bi8.npy',allow_pickle=True)
bi9=np.load('bi9.npy',allow_pickle=True)
fbi1=np.load('fbi1.npy',allow_pickle=True)
fbi2=np.load('fbi2.npy',allow_pickle=True)
fbi3=np.load('fbi3.npy',allow_pickle=True)
fbi4=np.load('fbi4.npy',allow_pickle=True)
fbi5=np.load('fbi5.npy',allow_pickle=True)
fbi6=np.load('fbi6.npy',allow_pickle=True)
fbi7=np.load('fbi7.npy',allow_pickle=True)
fbi8=np.load('fbi8.npy',allow_pickle=True)
fbi9=np.load('fbi9.npy',allow_pickle=True)

In [ ]:
a_b1= a_b1.tolist()
b_b1= b_b1.tolist()
c_b1= c_b1.tolist()
cp_b1= cp_b1.tolist()
ab_b1= ab_b1.tolist()
a_b2= a_b2.tolist()
b_b2= b_b2.tolist()
c_b2= c_b2.tolist()
cp_b2= cp_b2.tolist()
ab_b2= ab_b2.tolist()
a_b3= a_b3.tolist()
b_b3= b_b3.tolist()
c_b3= c_b3.tolist()
cp_b3= cp_b3.tolist()
ab_b3= ab_b3.tolist()
a_b4= a_b4.tolist()
b_b4= b_b4.tolist()
c_b4= c_b4.tolist()
cp_b4= cp_b4.tolist()
ab_b4= ab_b4.tolist()
a_b5= a_b5.tolist()
b_b5= b_b5.tolist()
c_b5= c_b5.tolist()
cp_b5= cp_b5.tolist()
ab_b5= ab_b5.tolist()
a_b6= a_b6.tolist()
b_b6= b_b6.tolist()
c_b6= c_b6.tolist()
cp_b6= cp_b6.tolist()
ab_b6= ab_b6.tolist()
a_b7= a_b7.tolist()
b_b7= b_b7.tolist()
c_b7= c_b7.tolist()
cp_b7= cp_b7.tolist()
ab_b7= ab_b7.tolist()
a_b8= a_b8.tolist()
b_b8= b_b8.tolist()
c_b8= c_b8.tolist()
cp_b8= cp_b8.tolist()
ab_b8= ab_b8.tolist()
a_b9= a_b9.tolist()
b_b9= b_b9.tolist()
c_b9= c_b9.tolist()
cp_b9= cp_b9.tolist()
ab_b9= ab_b9.tolist()

bi1=bi1.tolist()
bi2=bi2.tolist()
bi3=bi3.tolist()
bi4=bi4.tolist()
bi5=bi5.tolist()
bi6=bi6.tolist()
bi7=bi7.tolist()
bi8=bi8.tolist()
bi9=bi9.tolist()
fbi1=fbi1.tolist()
fbi2=fbi2.tolist()
fbi3=fbi3.tolist()
fbi4=fbi4.tolist()
fbi5=fbi5.tolist()
fbi6=fbi6.tolist()
fbi7=fbi7.tolist()
fbi8=fbi8.tolist()
fbi9=fbi9.tolist()

In [ ]:
a_b1=np.array(a_b1)
b_b1=np.array(b_b1)
c_b1=np.array(c_b1)
cp_b1=np.array(cp_b1)
ab_b1=np.array(ab_b1)
a_b2=np.array(a_b2)
b_b2=np.array(b_b2)
c_b2=np.array(c_b2)
cp_b2=np.array(cp_b2)
ab_b2=np.array(ab_b2)
a_b3=np.array(a_b3)
b_b3=np.array(b_b3)
c_b3=np.array(c_b3)
cp_b3=np.array(cp_b3)
ab_b3=np.array(ab_b3)
a_b4=np.array(a_b4)
b_b4=np.array(b_b4)
c_b4=np.array(c_b4)
cp_b4=np.array(cp_b4)
ab_b4=np.array(ab_b4)
a_b5=np.array(a_b5)
b_b5=np.array(b_b5)
c_b5=np.array(c_b5)
cp_b5=np.array(cp_b5)
ab_b5=np.array(ab_b5)
a_b6=np.array(a_b6)
b_b6=np.array(b_b6)
c_b6=np.array(c_b6)
cp_b6=np.array(cp_b6)
ab_b6=np.array(ab_b6)
a_b7=np.array(a_b7)
b_b7=np.array(b_b7)
c_b7=np.array(c_b7)
cp_b7=np.array(cp_b7)
ab_b7=np.array(ab_b7)
a_b8=np.array(a_b8)
b_b8=np.array(b_b8)
c_b8=np.array(c_b8)
cp_b8=np.array(cp_b8)
ab_b8=np.array(ab_b8)
a_b9=np.array(a_b9)
b_b9=np.array(b_b9)
c_b9=np.array(c_b9)
cp_b9=np.array(cp_b9)
ab_b9=np.array(ab_b9)
bi1=np.array(bi1)
bi2=np.array(bi2)
bi3=np.array(bi3)
bi4=np.array(bi4)
bi5=np.array(bi5)
bi6=np.array(bi6)
bi7=np.array(bi7)
bi8=np.array(bi8)
bi9=np.array(bi9)
fbi1=np.array(fbi1)
fbi2=np.array(fbi2)
fbi3=np.array(fbi3)
fbi4=np.array(fbi4)
fbi5=np.array(fbi5)
fbi6=np.array(fbi6)
fbi7=np.array(fbi7)
fbi8=np.array(fbi8)
fbi9=np.array(fbi9)

<ipython-input-8-2406efab5bf4>:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  a_b1=np.array(a_b1)
<ipython-input-8-2406efab5bf4>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a_b1=np.array(a_b1)
<ipython-input-8-2406efab5bf4>:4: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__arr

In [ ]:
bi_1=[None]*5000
for i in range(0,5000):
  bi_1[i]=bi1[i].item()
bi_2=[None]*5000
for i in range(0,5000):
  bi_2[i]=bi2[i].item()
bi_3=[None]*5000
for i in range(0,5000):
  bi_3[i]=bi3[i].item()
bi_4=[None]*5000
for i in range(0,5000):
  bi_4[i]=bi4[i].item()
bi_5=[None]*5000
for i in range(0,5000):
  bi_5[i]=bi5[i].item()
bi_6=[None]*5000
for i in range(0,5000):
  bi_6[i]=bi6[i].item()
bi_7=[None]*5000
for i in range(0,5000):
  bi_7[i]=bi7[i].item()
bi_8=[None]*5000
for i in range(0,5000):
  bi_8[i]=bi8[i].item()
bi_9=[None]*5000
for i in range(0,5000):
  bi_9[i]=bi9[i].item()


In [ ]:
import statistics

In [ ]:
fbi1=np.asarray(fbi1,dtype=np.float64)
fbi2=np.asarray(fbi2,dtype=np.float64)
fbi3=np.asarray(fbi3,dtype=np.float64)
fbi4=np.asarray(fbi4,dtype=np.float64)
fbi5=np.asarray(fbi5,dtype=np.float64)
fbi6=np.asarray(fbi6,dtype=np.float64)
fbi7=np.asarray(fbi7,dtype=np.float64)
fbi8=np.asarray(fbi8,dtype=np.float64)
fbi9=np.asarray(fbi9,dtype=np.float64)
a1=np.asarray(a_b1,dtype=np.float64)
a2=np.asarray(a_b2,dtype=np.float64)
a3=np.asarray(a_b3,dtype=np.float64)
a4=np.asarray(a_b4,dtype=np.float64)
a5=np.asarray(a_b5,dtype=np.float64)
a6=np.asarray(a_b6,dtype=np.float64)
a7=np.asarray(a_b7,dtype=np.float64)
a8=np.asarray(a_b8,dtype=np.float64)
a9=np.asarray(a_b9,dtype=np.float64)

In [ ]:
c=[None]*9
b=[None]*9
bi=[None]*9
a=[None]*9
fbi=[None]*9
c[0]=c_b1.mean()
b[0]=b_b1.mean()
bi[0]=statistics.mean(bi_1)
c[1]=c_b2.mean()
b[1]=b_b2.mean()
bi[1]=statistics.mean(bi_2)
c[2]=c_b3.mean()
b[2]=b_b3.mean()
bi[2]=statistics.mean(bi_3)
c[3]=c_b4.mean()
b[3]=b_b4.mean()
bi[3]=statistics.mean(bi_4)
c[4]=c_b5.mean()
b[4]=b_b5.mean()
bi[4]=statistics.mean(bi_5)
c[5]=c_b6.mean()
b[5]=b_b6.mean()
bi[5]=statistics.mean(bi_6)
c[6]=c_b7.mean()
b[6]=b_b7.mean()
bi[6]=statistics.mean(bi_7)
c[7]=c_b8.mean()
b[7]=b_b8.mean()
bi[7]=statistics.mean(bi_8)
c[8]=c_b9.mean()
b[8]=b_b9.mean()
bi[8]=statistics.mean(bi_9)
a[0]=a1.mean()
a[1]=a2.mean()
a[2]=a3.mean()
a[3]=a4.mean()
a[4]=a5.mean()
a[5]=a6.mean()
a[6]=a7.mean()
a[7]=a8.mean()
a[8]=a9.mean()
fbi[0]=fbi1.mean()
fbi[1]=fbi2.mean()
fbi[2]=fbi3.mean()
fbi[3]=fbi4.mean()
fbi[4]=fbi5.mean()
fbi[5]=fbi6.mean()
fbi[6]=fbi7.mean()
fbi[7]=fbi8.mean()
fbi[8]=fbi9.mean()


In [ ]:
print(bi[8],c[8],b[8],fbi[8],a[8])
m1= fbi[0] + a[0]*x
m2= fbi[1] + a[1]*x
m3= fbi[2] + a[2]*x
m4= fbi[3] + a[3]*x
m5= fbi[4] + a[4]*x
m6= fbi[5] + a[5]*x
m7= fbi[6] + a[6]*x
m8= fbi[7] + a[7]*x
m9= fbi[8] + a[8]*x

-0.02254592673468833 0.2964075124271447 -0.39371056626457723 0.033380840278198105 0.4180577221509069


In [ ]:
y_h1=bi[0]+c[0]*x+b[0]*m1
y_h2=bi[1]+c[1]*x+b[1]*m2
y_h3=bi[2]+c[2]*x+b[2]*m3
y_h4=bi[3]+c[3]*x+b[3]*m4
y_h5=bi[4]+c[4]*x+b[4]*m5
y_h6=bi[5]+c[5]*x+b[5]*m6
y_h7=bi[6]+c[6]*x+b[6]*m7
y_h8=bi[7]+c[7]*x+b[7]*m8
y_h9=bi[8]+c[8]*x+b[8]*m9


In [ ]:
TSS = ((y-y.mean())**2).sum(axis=1).sum()
SSR1 = ((y_h1-y.mean())**2).sum(axis=1).sum()
SSE1 = ((y-y_h1)**2).sum(axis=1).sum()
SSR2 = ((y_h2-y.mean())**2).sum(axis=1).sum()
SSE2 = ((y-y_h2)**2).sum(axis=1).sum()
SSR3 = ((y_h3-y.mean())**2).sum(axis=1).sum()
SSE3 = ((y-y_h3)**2).sum(axis=1).sum()
SSR4 = ((y_h4-y.mean())**2).sum(axis=1).sum()
SSE4 = ((y-y_h4)**2).sum(axis=1).sum()
SSR5 = ((y_h5-y.mean())**2).sum(axis=1).sum()
SSE5 = ((y-y_h5)**2).sum(axis=1).sum()
SSR6 = ((y_h6-y.mean())**2).sum(axis=1).sum()
SSE6 = ((y-y_h6)**2).sum(axis=1).sum()
SSR7 = ((y_h7-y.mean())**2).sum(axis=1).sum()
SSE7 = ((y-y_h7)**2).sum(axis=1).sum()
SSR8 = ((y_h8-y.mean())**2).sum(axis=1).sum()
SSE8 = ((y-y_h8)**2).sum(axis=1).sum()
SSR9 = ((y_h9-y.mean())**2).sum(axis=1).sum()
SSE9 = ((y-y_h9)**2).sum(axis=1).sum()


In [ ]:
MSE1 = np.sqrt(((y-y_h1)**2).sum(axis=1).sum()/60)
MSE2 = np.sqrt(((y-y_h2)**2).sum(axis=1).sum()/60)
MSE3 = np.sqrt(((y-y_h3)**2).sum(axis=1).sum()/60)
MSE4 = np.sqrt(((y-y_h4)**2).sum(axis=1).sum()/60)
MSE5 = np.sqrt(((y-y_h5)**2).sum(axis=1).sum()/60)
MSE6 = np.sqrt(((y-y_h6)**2).sum(axis=1).sum()/60)
MSE7 = np.sqrt(((y-y_h7)**2).sum(axis=1).sum()/60)
MSE8 = np.sqrt(((y-y_h8)**2).sum(axis=1).sum()/60)
MSE9 = np.sqrt(((y-y_h9)**2).sum(axis=1).sum()/60)


In [ ]:
print(MSE1)
print(MSE2)
print(MSE3)
print(MSE4)
print(MSE5)
print(MSE6)
print(MSE7)
print(MSE8)
print(MSE9)


0.9013448431257333
0.8973784318419643
0.8969238493241668
0.8968009356159277
0.89677298187407
0.8967747537871957
0.8967839806281416
0.8967928283908538
0.896798901158996


In [ ]:
MSA1 = np.abs(y-y_h1).sum(axis=1).sum()/60
MSA2 = np.abs(y-y_h2).sum(axis=1).sum()/60
MSA3 = np.abs(y-y_h3).sum(axis=1).sum()/60
MSA4 = np.abs(y-y_h4).sum(axis=1).sum()/60
MSA5 = np.abs(y-y_h5).sum(axis=1).sum()/60
MSA6 = np.abs(y-y_h6).sum(axis=1).sum()/60
MSA7 = np.abs(y-y_h7).sum(axis=1).sum()/60
MSA8 = np.abs(y-y_h8).sum(axis=1).sum()/60
MSA9 = np.abs(y-y_h9).sum(axis=1).sum()/60

In [ ]:
print(MSA1)
print(MSA2)
print(MSA3)
print(MSA4)
print(MSA5)
print(MSA6)
print(MSA7)
print(MSA8)
print(MSA9)

1.191001911071368
1.1886223439302488
1.190540570939812
1.1919557969070784
1.1928483729595294
1.193542377631005
1.1939736924181141
1.1942301768929076
1.1943659525855195
